# 🤖 Análisis de Riesgo de Automatización Laboral por IA

## Modelo Predictivo - Jalisco, México (2025-2030)

**Autor:** Carlos Pulido Rosas  
**Institución:** CUCEA - Universidad de Guadalajara  
**Programa:** Maestría en Ciencia de los Datos  
**Línea LGAC:** SMART DATA  
**Fecha:** Noviembre 2025

---

### 📋 Objetivo del Proyecto

Desarrollar un modelo predictivo usando **PySpark** que identifique ocupaciones con alto riesgo de sustitución laboral por Inteligencia Artificial en el estado de Jalisco, mediante análisis de características ocupacionales y tendencias socioeconómicas.

### 📊 Fuentes de Datos

1. **O*NET Database** - Características de ocupaciones
2. **INEGI - ENOE** - Empleo en Jalisco
3. **Estudios de automatización** (Frey-Osborne, McKinsey)

### 🎯 Resultados Esperados

- Identificación de ocupaciones de alto riesgo
- Cuantificación del impacto laboral por sector
- Proyecciones 2025-2030
- Recomendaciones de política pública

## 📑 Tabla de Contenidos

1. [Configuración Inicial](#1-configuración-inicial)
2. [Carga de Datos](#2-carga-de-datos)
3. [Exploración de Datos](#3-exploración-de-datos)
4. [Preprocesamiento](#4-preprocesamiento)
5. [Feature Engineering](#5-feature-engineering)
6. [Análisis de Riesgo](#6-análisis-de-riesgo)
7. [Análisis por Dimensiones](#7-análisis-por-dimensiones)
8. [Visualizaciones](#8-visualizaciones)
9. [Modelado Predictivo](#9-modelado-predictivo)
10. [Resultados y Conclusiones](#10-resultados-y-conclusiones)

---

## 1. Configuración Inicial

### 1.1 Importación de Librerías

In [ ]:
# Suprimir warnings
import warnings
warnings.filterwarnings('ignore')

# Librerías básicas
import sys
import os

# Agregar src al path
sys.path.append('../src')

# PySpark
from pyspark.sql import SparkSession
import pyspark.pandas as ps

# Análisis de datos
import pandas as pd
import numpy as np

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Configurar visualizaciones
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("✓ Librerías importadas exitosamente")

### 1.2 Crear Spark Session

In [ ]:
from data_loader import create_spark_session

# Crear sesión de Spark
spark = create_spark_session(
    app_name="AI_Automation_Risk_Jalisco",
    memory="8g"
)

print(f"✓ Spark Session creada")
print(f"  Versión: {spark.version}")
print(f"  Master: {spark.sparkContext.master}")

---

## 2. Carga de Datos

### 2.1 Datos Simulados para Pruebas

Para este análisis usaremos datos simulados que replican las características del Global Terrorism Database adaptado a ocupaciones laborales.

In [ ]:
from data_loader import load_sample_data, convert_to_pandas_api

# Generar datos simulados
# Para producción: usar load_onet_occupations() y load_enoe_jalisco()
df_spark = load_sample_data(spark, n_occupations=200)

print(f"✓ Datos cargados: {df_spark.count():,} ocupaciones")

### 2.2 Convertir a pyspark.pandas

Convertimos a `pyspark.pandas` para facilitar la manipulación con sintaxis familiar.

In [ ]:
# Convertir a pyspark.pandas
df = convert_to_pandas_api(df_spark)

print(f"✓ Conversión exitosa")
print(f"  Shape: {df.shape}")
print(f"  Tipo: {type(df)}")

---

## 3. Exploración de Datos

### 3.1 Primeras Filas

In [ ]:
# Mostrar primeras 10 ocupaciones
df.head(10)

### 3.2 Información General del Dataset

In [ ]:
# Información del dataset
print("="*80)
print("INFORMACIÓN DEL DATASET")
print("="*80)
print(f"\nDimensiones: {df.shape}")
print(f"Filas: {df.shape[0]:,}")
print(f"Columnas: {df.shape[1]}")

print("\nColumnas disponibles:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i:2d}. {col}")

### 3.3 Estadísticas Descriptivas

In [ ]:
# Estadísticas de variables numéricas clave
numeric_cols = ['routine_index', 'cognitive_demand', 'social_interaction', 
                'creativity', 'education_level', 'workers_jalisco', 'avg_salary_mxn']

stats = df[numeric_cols].describe()
print("\nESTADÍSTICAS DESCRIPTIVAS")
print("="*80)
print(stats)

### 3.4 Distribución por Sector

In [ ]:
# Distribución de ocupaciones por sector
print("\nDISTRIBUCIÓN POR SECTOR ECONÓMICO")
print("="*80)
sector_counts = df['sector'].value_counts()
print(sector_counts)

# Visualización
plt.figure(figsize=(10, 6))
sector_counts.plot(kind='bar', color='steelblue', edgecolor='black')
plt.xlabel('Sector Económico', fontsize=12)
plt.ylabel('Número de Ocupaciones', fontsize=12)
plt.title('Distribución de Ocupaciones por Sector', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

### 3.5 Valores Faltantes

In [ ]:
from data_preprocessing import analyze_missing_values

# Analizar valores faltantes
missing_df = analyze_missing_values(df)

if len(missing_df) > 0:
    print("\nCOLUMNAS CON VALORES FALTANTES:")
    print("="*80)
    print(missing_df)
else:
    print("\n✓ No hay valores faltantes en el dataset")

---

## 4. Preprocesamiento

### 4.1 Pipeline de Limpieza

In [ ]:
from data_preprocessing import preprocess_pipeline

# Ejecutar pipeline de preprocesamiento
df_clean = preprocess_pipeline(df, config={
    'handle_missing': 'auto',
    'remove_duplicates': True,
    'filter_outliers': False
})

print(f"\n✓ Preprocesamiento completado")
print(f"  Filas originales: {df.shape[0]:,}")
print(f"  Filas limpias: {df_clean.shape[0]:,}")
print(f"  Diferencia: {df.shape[0] - df_clean.shape[0]:,} filas eliminadas")

### 4.2 Validación de Calidad

In [ ]:
from data_preprocessing import validate_data_quality

# Validar calidad de datos
metrics = validate_data_quality(df_clean)

print("\nMÉTRICAS DE CALIDAD DE DATOS")
print("="*80)
for key, value in metrics.items():
    print(f"  {key}: {value}")

---

## 5. Feature Engineering

### 5.1 Creación de Features

Crearemos múltiples índices y features derivados para el análisis.

In [ ]:
from feature_engineering import feature_engineering_pipeline

# Ejecutar pipeline de feature engineering
df_features = feature_engineering_pipeline(df_clean, config={
    'create_indices': True,
    'create_categories': True,
    'create_ratios': True,
    'create_temporal': True,
    'create_sector_agg': True
})

print(f"\n✓ Feature engineering completado")
print(f"  Columnas originales: {df_clean.shape[1]}")
print(f"  Columnas finales: {df_features.shape[1]}")
print(f"  Features nuevos: {df_features.shape[1] - df_clean.shape[1]}")

### 5.2 Features Creados

**Índices Base:**
- `routine_index` - Grado de rutinización (0-100)
- `cognitive_demand` - Demanda cognitiva (0-100)
- `social_interaction` - Nivel de interacción social (0-100)
- `creativity` - Nivel de creatividad requerida (0-100)
- `complexity_index` - Complejidad general
- `automation_susceptibility` - Susceptibilidad a automatización

**Categorías:**
- `education_category` - Básica/Media/Superior/Posgrado
- `salary_category` - Quintiles salariales
- `occupation_size` - Pequeña/Mediana/Grande

**Features Temporales:**
- `risk_projection_2026` a `risk_projection_2030`

**Agregaciones por Sector:**
- `sector_avg_risk` - Riesgo promedio del sector
- `sector_total_workers` - Total de trabajadores del sector

In [ ]:
# Mostrar algunos features creados
print("\nNUEVOS FEATURES CREADOS:")
print("="*80)
new_cols = set(df_features.columns) - set(df_clean.columns)
for i, col in enumerate(sorted(new_cols), 1):
    print(f"  {i:2d}. {col}")

---

## 6. Análisis de Riesgo de Automatización

### 6.1 Calcular Riesgo Base

Usando metodología basada en Frey & Osborne (2013).

In [ ]:
from automation_analyzer import AutomationRiskAnalyzer

# Crear analizador
analyzer = AutomationRiskAnalyzer()

# Calcular riesgo de automatización
df_risk = analyzer.calculate_automation_risk(
    df_features,
    method='frey_osborne'
)

print("\n✓ Riesgo de automatización calculado")
print("\nDISTRIBUCIÓN DE RIESGO:")
print("="*80)
print(df_risk['automation_risk'].describe())

### 6.2 Categorizar Riesgo

Clasificamos ocupaciones en tres categorías:
- **Bajo Riesgo:** < 0.30
- **Riesgo Medio:** 0.30 - 0.70
- **Alto Riesgo:** > 0.70

In [ ]:
# Categorizar riesgo
df_risk = analyzer.categorize_risk(df_risk, thresholds=(0.30, 0.70))

print("\nCATEGORIZACIÓN DE RIESGO")
print("="*80)
risk_counts = df_risk['risk_category'].value_counts()
print("\nConteo:")
print(risk_counts)

print("\nPorcentajes:")
risk_pct = df_risk['risk_category'].value_counts(normalize=True) * 100
print(risk_pct)

### 6.3 Top Ocupaciones en Riesgo

In [ ]:
# Top 20 ocupaciones más riesgosas
top_risk = analyzer.identify_top_at_risk(df_risk, n=20)

print("\nTOP 20 OCUPACIONES CON MAYOR RIESGO")
print("="*80)
print(top_risk[['occupation_name', 'sector', 'automation_risk', 'workers_jalisco']])

### 6.4 Ocupaciones Más Seguras

In [ ]:
# Top 20 ocupaciones más seguras
low_risk = analyzer.identify_low_risk_occupations(df_risk, n=20)

print("\nTOP 20 OCUPACIONES MÁS SEGURAS")
print("="*80)
print(low_risk[['occupation_name', 'sector', 'automation_risk']])

---

## 7. Análisis por Dimensiones

### 7.1 Análisis por Sector Económico

In [ ]:
# Análisis por sector
sector_analysis = analyzer.analyze_by_sector(df_risk)

print("\nANÁLISIS DE RIESGO POR SECTOR")
print("="*80)
print(sector_analysis.sort_values('risk_mean', ascending=False))

### 7.2 Análisis por Nivel Educativo

In [ ]:
# Análisis por educación
education_analysis = analyzer.analyze_by_education(df_risk)

print("\nANÁLISIS DE RIESGO POR NIVEL EDUCATIVO")
print("="*80)
print(education_analysis)

### 7.3 Impacto Económico

In [ ]:
# Calcular impacto económico
impact = analyzer.calculate_economic_impact(df_risk)

print("\nIMPACTO ECONÓMICO DE LA AUTOMATIZACIÓN")
print("="*80)
print(f"\nTotal de trabajadores: {impact['total_workers']:,}")
print(f"Trabajadores en alto riesgo: {impact['workers_high_risk']:,}")
print(f"Porcentaje en riesgo: {impact['pct_workers_at_risk']:.1f}%")

if impact['salary_at_risk_mxn']:
    print(f"\nMasa salarial en riesgo: ${impact['salary_at_risk_mxn']:,.2f} MXN")
    print(f"Porcentaje de masa salarial: {impact['pct_salary_at_risk']:.1f}%")

---

## 8. Visualizaciones

### 8.1 Distribución del Riesgo

In [ ]:
from visualizations import plot_risk_distribution

# Gráfico de distribución
plot_risk_distribution(df_risk)

### 8.2 Riesgo por Sector

In [ ]:
from visualizations import plot_risk_by_sector

# Riesgo por sector
plot_risk_by_sector(df_risk, top_n=15)

### 8.3 Salario vs Riesgo (Interactivo)

In [ ]:
from visualizations import plot_salary_vs_risk

# Scatter plot interactivo
plot_salary_vs_risk(df_risk)

### 8.4 Educación vs Riesgo

In [ ]:
from visualizations import plot_education_vs_risk

# Boxplot por nivel educativo
plot_education_vs_risk(df_risk)

### 8.5 Heatmap: Sector vs Educación

In [ ]:
from visualizations import plot_heatmap_sector_education

# Mapa de calor
plot_heatmap_sector_education(df_risk)

### 8.6 Treemap: Trabajadores en Riesgo

In [ ]:
from visualizations import plot_treemap_workers_at_risk

# Treemap interactivo
plot_treemap_workers_at_risk(df_risk)

### 8.7 Proyecciones Temporales 2025-2030

In [ ]:
from visualizations import plot_temporal_projections

# Serie temporal de proyecciones
plot_temporal_projections(df_risk)

### 8.8 Matriz de Correlación

In [ ]:
from visualizations import plot_correlation_matrix

# Correlaciones entre variables
plot_correlation_matrix(df_risk)

### 8.9 Top Ocupaciones en Riesgo

In [ ]:
from visualizations import plot_top_occupations_at_risk

# Barras de top ocupaciones
plot_top_occupations_at_risk(df_risk, n=20)

---

## 9. Modelado Predictivo

### 9.1 Preparar Datos para Modelado

In [ ]:
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml import Pipeline

# Convertir a Spark DataFrame para MLlib
df_spark_ml = df_risk.to_spark()

# Features para el modelo
feature_cols = [
    'routine_index',
    'cognitive_demand',
    'social_interaction',
    'creativity',
    'education_level',
    'avg_salary_mxn'
]

# Ensamblar features
assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol='features',
    handleInvalid='skip'
)

# Indexar target (risk_category)
indexer = StringIndexer(
    inputCol='risk_category',
    outputCol='label'
)

print("✓ Pipeline de features configurado")

### 9.2 Split Train/Test

In [ ]:
# Dividir datos 80/20
train_df, test_df = df_spark_ml.randomSplit([0.8, 0.2], seed=42)

print(f"✓ Datos divididos:")
print(f"  Entrenamiento: {train_df.count():,} registros")
print(f"  Prueba: {test_df.count():,} registros")

### 9.3 Entrenar Random Forest

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

# Crear modelo Random Forest
rf = RandomForestClassifier(
    featuresCol='features',
    labelCol='label',
    numTrees=100,
    maxDepth=10,
    seed=42
)

# Pipeline completo
pipeline = Pipeline(stages=[assembler, indexer, rf])

# Entrenar
print("Entrenando modelo Random Forest...")
model = pipeline.fit(train_df)

print("✓ Modelo entrenado exitosamente")

### 9.4 Evaluación del Modelo

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Hacer predicciones
predictions = model.transform(test_df)

# Evaluar
evaluator = MulticlassClassificationEvaluator(
    labelCol='label',
    predictionCol='prediction',
    metricName='accuracy'
)

print("\nMÉTRICAS DEL MODELO")
print("="*80)

# Accuracy
accuracy = evaluator.evaluate(predictions)
print(f"Accuracy: {accuracy:.3f}")

# Otras métricas
for metric in ['weightedPrecision', 'weightedRecall', 'f1']:
    evaluator.setMetricName(metric)
    score = evaluator.evaluate(predictions)
    print(f"{metric}: {score:.3f}")

### 9.5 Feature Importance

In [ ]:
# Obtener importancia de features
rf_model = model.stages[-1]
importances = rf_model.featureImportances

# Crear DataFrame
feature_importance_df = ps.DataFrame({
    'feature': feature_cols,
    'importance': importances.toArray()
}).sort_values('importance', ascending=False)

print("\nIMPORTANCIA DE FEATURES")
print("="*80)
print(feature_importance_df)

# Visualizar
plt.figure(figsize=(10, 6))
plt.barh(feature_importance_df['feature'], feature_importance_df['importance'], color='coral', edgecolor='black')
plt.xlabel('Importancia', fontsize=12)
plt.title('Importancia de Características en el Modelo', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

---

## 10. Resultados y Conclusiones

### 10.1 Hallazgos Clave

In [ ]:
print("\n" + "="*80)
print("HALLAZGOS CLAVE DEL ANÁLISIS")
print("="*80)

# 1. Distribución general
total_high = len(df_risk[df_risk['automation_risk'] >= 0.70])
pct_high = (total_high / len(df_risk)) * 100

print(f"\n1. DISTRIBUCIÓN DE RIESGO")
print(f"   • {pct_high:.1f}% de ocupaciones en alto riesgo")
print(f"   • {len(df_risk[df_risk['automation_risk'] < 0.30]):,} ocupaciones seguras")

# 2. Sector más afectado
top_sector = sector_analysis.iloc[0]
print(f"\n2. SECTOR MÁS AFECTADO")
print(f"   • {top_sector['sector']}")
print(f"   • Riesgo promedio: {top_sector['risk_mean']:.3f}")
print(f"   • Trabajadores: {top_sector['total_workers']:,}")

# 3. Correlación educación-riesgo
corr = df_risk[['education_level', 'automation_risk']].corr().iloc[0, 1]
print(f"\n3. EDUCACIÓN vs RIESGO")
print(f"   • Correlación: {corr:.3f}")
if corr < -0.3:
    print(f"   • ✓ Mayor educación → Menor riesgo")
else:
    print(f"   • ⚠ Relación débil")

# 4. Impacto económico
print(f"\n4. IMPACTO ECONÓMICO")
print(f"   • Trabajadores en riesgo: {impact['workers_high_risk']:,} ({impact['pct_workers_at_risk']:.1f}%)")
if impact['salary_at_risk_mxn']:
    print(f"   • Masa salarial en riesgo: ${impact['salary_at_risk_mxn']:,.2f} MXN")

# 5. Precisión del modelo
print(f"\n5. DESEMPEÑO DEL MODELO")
print(f"   • Accuracy: {accuracy:.3f}")
print(f"   • Feature más importante: {feature_importance_df.iloc[0]['feature']}")

### 10.2 Recomendaciones de Política Pública

**Basadas en los hallazgos del análisis:**

#### 1. **Sectores Prioritarios para Intervención**

Los sectores con mayor riesgo requieren programas de reconversión laboral inmediatos.

#### 2. **Programas de Capacitación**

Enfocar la capacitación en:
- Pensamiento crítico y creativo
- Inteligencia emocional
- Habilidades sociales
- Resolución de problemas complejos

#### 3. **Inversión en Educación**

La correlación entre educación y riesgo sugiere que la educación superior es un factor protector.

#### 4. **Monitoreo Continuo**

Establecer un sistema de monitoreo para actualizar proyecciones conforme avanza la adopción de IA.

#### 5. **Apoyo a Trabajadores**

Programas de apoyo económico temporal para trabajadores en transición laboral.

### 10.3 Limitaciones del Estudio

1. **Datos simulados:** Este análisis usa datos simulados. En producción se requieren datos reales de O*NET y ENOE.

2. **Alcance geográfico:** Limitado a Jalisco. Resultados pueden variar en otras entidades.

3. **Proyecciones:** Basadas en tendencias actuales. Cambios disruptivos pueden alterar predicciones.

4. **Factores externos:** No considera políticas gubernamentales futuras o cambios económicos globales.

### 10.4 Trabajo Futuro

- **Datos reales:** Integrar datos completos de O*NET y ENOE
- **Análisis temporal:** Incorporar series de tiempo históricas
- **Análisis geográfico:** Desglose por municipios de Jalisco
- **Modelos avanzados:** Probar Deep Learning y ensembles
- **Validación externa:** Contrastar con estudios internacionales

---

## 📊 Resumen Ejecutivo

### Resultados Principales

✅ **{pct_high:.1f}%** de ocupaciones analizadas tienen alto riesgo de automatización

✅ **{impact['workers_high_risk']:,}** trabajadores en Jalisco están en alto riesgo

✅ **{top_sector['sector']}** es el sector más afectado

✅ Modelo predictivo alcanza **{accuracy:.1f}%** de precisión

### Siguientes Pasos

1. Presentar resultados a stakeholders
2. Implementar con datos reales
3. Desarrollar dashboard interactivo
4. Publicar hallazgos académicos

---

## 📞 Contacto

**Carlos Pulido Rosas**  
📧 carlos.pulido.rosas@gmail.com  
📱 +52 33 1030 5580  
🎓 CUCEA - Universidad de Guadalajara  
🔗 [GitHub](https://github.com/Carpuro/ai-automation-risk-jalisco)

---

## 📚 Referencias

1. Frey, C. B., & Osborne, M. A. (2017). *The future of employment: How susceptible are jobs to computerisation?* Technological Forecasting and Social Change, 114, 254-280.

2. McKinsey Global Institute (2023). *A Future That Works: Automation, Employment, and Productivity.*

3. O*NET Program (2024). *Occupational Information Network.* U.S. Department of Labor.

4. INEGI (2024). *Encuesta Nacional de Ocupación y Empleo (ENOE).*

---

**Fin del Análisis** ✅

*Notebook generado: Noviembre 2025*  
*Versión: 1.0*